## Task
1. Identify all shops that are deemed to have conducted order brushing.
2. For each shop that is identified to have conducted order brushing, identify the buyers suspected to have conducted order brushing for that shop.

Definition of order brushing
- concentration_rate >= 3
- concentration_rate = num_orders_1hr / num_unique_buyer_1hr
- **suspicious buyers** are deemed to be the buyer that contributed the highest proportion of orders to a shop

In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [157]:
df = pd.read_csv('order_brush_order.csv')

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   orderid     222750 non-null  int64 
 1   shopid      222750 non-null  int64 
 2   userid      222750 non-null  int64 
 3   event_time  222750 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


In [159]:
df.describe()

,orderid,shopid,userid
count,2.227500e+05,2.227500e+05,2.227500e+05
mean,3.130027e+13,9.433117e+07,9.802880e+07
std,1.222774e+11,5.695790e+07,6.839048e+07
min,3.107520e+13,1.000900e+04,1.000700e+04
25%,3.120360e+13,4.980267e+07,3.508127e+07
50%,3.130561e+13,9.033636e+07,9.309625e+07
75%,3.140604e+13,1.475053e+08,1.590612e+08
max,3.150720e+13,2.154352e+08,2.155262e+08


In [160]:
df.event_time.min(), df.event_time.max()

('2019-12-27 00:00:00', '2019-12-31 23:59:56')

In [161]:
df['event_time'] = df['event_time'].astype('datetime64[ns]')
df = df.set_index('event_time').sort_index()

### Trying to define whether order brushing has occurred

> Detect instataneous concentration rate spikes

In [162]:
df.head()

,orderid,shopid,userid
event_time,,,
2019-12-27 00:00:00,31075200506751,6042309,97707522
2019-12-27 00:00:00,31075200506752,104804492,97707522
2019-12-27 00:00:00,31075200506753,8715449,97707522
2019-12-27 00:00:02,31075201870570,190969466,170182475
2019-12-27 00:00:05,31075205798264,2859407,12532131


In [163]:
def nunique(arr):
    return len(set(arr))

orders_shop_1h = df.groupby('shopid').rolling('1h', min_periods=3).orderid.count()
buyers_shop_1h = df.groupby('shopid').rolling('1h', min_periods=3).userid.apply(nunique)

In [164]:
orders_shop_1h

shopid     event_time         
10009      2019-12-27 03:06:50   NaN
10051      2019-12-27 19:16:11   NaN
           2019-12-29 01:56:19   NaN
10061      2019-12-28 09:27:55   NaN
           2019-12-28 12:05:32   NaN
                                  ..
214949521  2019-12-31 20:06:43   NaN
214964814  2019-12-29 22:26:16   NaN
215175775  2019-12-31 09:06:31   NaN
           2019-12-31 14:14:37   NaN
215435223  2019-12-31 18:34:56   NaN
Name: orderid, Length: 222750, dtype: float64

In [165]:
concentration = orders_shop_1h / buyers_shop_1h
concentration_spikes = concentration >= 2
concentration_events = concentration_spikes.index[concentration_spikes].values

In [166]:
concentration_events

array([(10159, Timestamp('2019-12-29 20:56:43')),
       (10159, Timestamp('2019-12-29 21:36:11')),
       (10402, Timestamp('2019-12-28 14:30:26')),
       (10536, Timestamp('2019-12-27 11:44:08')),
       (10536, Timestamp('2019-12-27 17:55:26')),
       (29583, Timestamp('2019-12-27 23:25:28')),
       (29583, Timestamp('2019-12-27 23:27:59')),
       (42472, Timestamp('2019-12-27 22:38:38')),
       (42818, Timestamp('2019-12-31 10:57:26')),
       (42818, Timestamp('2019-12-31 10:59:41')),
       (42818, Timestamp('2019-12-31 11:03:57')),
       (76934, Timestamp('2019-12-28 19:51:47')),
       (76934, Timestamp('2019-12-28 19:53:41')),
       (195531, Timestamp('2019-12-29 20:39:18')),
       (195531, Timestamp('2019-12-29 21:37:31')),
       (425364, Timestamp('2019-12-31 13:02:32')),
       (599533, Timestamp('2019-12-31 18:24:00')),
       (605561, Timestamp('2019-12-28 19:11:01')),
       (722132, Timestamp('2019-12-27 14:29:42')),
       (731606, Timestamp('2019-12-27 13:31:

In [167]:
events = []

for shopid, timestamp in concentration_events:
    event_start = timestamp - pd.Timedelta('1 hours')
    event_end = timestamp
    event = df[(df['shopid'] == shopid) & (df.index >= event_start) & (df.index <= event_end)]
    events.append(event)
    
df_events = pd.concat(events)

df_events.shape

(2469, 3)

In [168]:
df_events.head()

,orderid,shopid,userid
event_time,,,
2019-12-29 20:17:14,31321034505684,10159,1138092
2019-12-29 20:38:09,31322289392454,10159,214988798
2019-12-29 20:40:54,31322453873412,10159,214988798
2019-12-29 20:56:43,31323403482068,10159,214988798
2019-12-29 20:38:09,31322289392454,10159,214988798


In [169]:
def extract_common_value(x):
    mode_list = x.mode()
    if len(mode_list) <= 1:
        return mode_list[0]
    else:
        return '&'.join([str(c) for c in sorted(mode_list)])

In [170]:
order_brush = df_events.groupby('shopid')['userid'].apply(extract_common_value)
order_brush = order_brush.to_frame()

In [171]:
order_brush.sample(10)

,userid
shopid,
64369518,172591713
105378074,207428158
9641401,43131979
200773571,638190&185389828
189308408,27456547
145146309,111115654
126261266,76441165
123401085,12869645
4888564,143847348


In [172]:
order_brush.shape

(353, 1)

### Preparing for Submission

In [173]:
def make_new_submission(num, order_brush):
    shops_all = df['shopid'].unique()

    submission = pd.DataFrame({'shopid': shops_all, 'userid': np.zeros(len(shops_all))})

    submission.userid = submission.userid.astype('int')
    submission.set_index('shopid', inplace=True)
    submission.update(order_brush)
    submission = submission.reset_index()
    
    submission.to_csv('submission{}.csv'.format(num), index=False)
    return submission

In [174]:
new_submission = make_new_submission(9, order_brush)

new_submission.shape

(18770, 2)